### Initiialise script

In [1]:
import numpy as np
import os
import glob
import pandas as pd 
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

def make_antecedent_conditions (results, timeperiod, statistic, days):
    # Method to get list of dates (is not working with ReFH2)
    # dates = []
    # for day in range(days,0,-1):
    #   dates.append(start_of_event - (oneday*day))
    
    # Get a list of dates (for some reason when create within Python, ReFH2 doesn't accept them)
    # Use an existing csv which has dates which work
    toedit = pd.read_csv("LinDykeAntecedentConditions/wholeyear/FormattedDates.csv", header=None)
    times = toedit[len(toedit)-days:][0].values
    
    # Create dataframe with results
    df = pd.DataFrame({"Times":times, 'Rainfall':results[statistic]})
    # Save to csv    
    df.to_csv("LinDykeAntecedentConditions/{}/lindyke_daily_antecedent_conditions_{}_{}days.csv".format(timeperiod, statistic, days),
                    header = False,index= False, date_format='%Y/%m/%d %H:%M')
    return df

In [2]:
timeperiod = 'jja' #jja #wholeyear

### Load the times related to the hourly data for each catchment grid cell 

In [3]:
times = np.load('LinDykeRainfall/{}/times.npy'.format(timeperiod), allow_pickle=True)

### Get a list of all the individual grid cell .npy files
(Don't include files containing times and the total)

In [5]:
if timeperiod == 'wholeyear':
    npy_files = glob.glob('LinDykeRainfall/wholeyear/*.npy')[2:]
elif timeperiod == 'jja':
    npy_files = glob.glob('LinDykeRainfall/jja/*.npy')[:-2]

### Create an array containing all the daily values for the grid cells in the catchment in one place

In [76]:
# Create an array to store the daily values for all the grid cells in the catchment
all_daily_sums = np. array([])

# Add the daily data for each grid cell
for file in npy_files:
    one_cell = np.load(file, allow_pickle = True)
    # Convert to a dataframe with the dates attached 
    df = pd.DataFrame({'Dates': pd.to_datetime(times), 'Precipitation': one_cell})
    # Resample from horus to days
    daily_sums = df.set_index('Dates').resample('D')['Precipitation'].sum()
    # Resampling process adds back in the days which are in the months we wanted to exclude
    daily_sums_df =pd.DataFrame(daily_sums)
    daily_sums_df.reset_index(inplace=True)
    # Filter back so we just have the months that we want to keep (JJA)
    daily_sums_df= daily_sums_df[daily_sums_df['Dates'].dt.month.isin([6,7,8])]
      
    # Join to array containing data from across all the cells
    all_daily_sums = np.append(all_daily_sums, daily_sums_df['Precipitation'])

### Find the mean and Xth percentile daily precipitation values across the area covering the catchment 

In [85]:
stats = ["25th Percentile", '50th Percentile','75th Percentile',
         '88th Percentile', '90th Percentile','95th Percentile', '99th Percentile', 'mean']

results = {}
for stat in stats.copy():
    if stat == 'mean':
        results[str(stat)] = np.mean(all_daily_sums)
    else:
        results[str(stat)] = np.percentile(all_daily_sums, int(stat[0:2]))
results    

{'25th Percentile': 0.0,
 '50th Percentile': 0.10000000149011612,
 '75th Percentile': 1.7999999523162842,
 '88th Percentile': 5.0,
 '90th Percentile': 5.900000095367432,
 '95th Percentile': 9.899999618530273,
 '99th Percentile': 23.299999237060547,
 'mean': 1.954647602511898}

### Save results to file

In [86]:
# Define when the event will start
start_of_event = datetime(2022, 8, 1, 0, 0)
oneday = timedelta(days=1)
for statistic in stats:
    for days in [4,365,15]:
        rainfall = make_antecedent_conditions(results, timeperiod,  statistic,days)